In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.TxtinoutReader import TxtinoutReader
from pathlib import Path
import os
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

In [3]:
contaminant = 'Diuron'
conca = 'fluvia'

In [4]:
cwd = Path(os.path.abspath(''))

In [5]:
reader = TxtinoutReader(cwd / 'data' / 'txtinouts' / 'TxtInOut_fluvia_2')

In [6]:
#set up
reader.set_beginning_and_end_year(2000, 2022)
reader.enable_object_in_print_prt('channel_sd', True, False, False, False)
reader.enable_object_in_print_prt('poll', True, False, False, False)
reader.disable_csv_print()

#delete pollutants that are not the current one
file = reader.register_file('pollutants.def', has_units = False, index = 'name')
df = file.df
df = df[df['name'] == contaminant]
file.df = df.copy()
file.overwrite_file()

file = reader.register_file('pollutants_om.exc', has_units = False)
df = file.df
df = df[df['pollutants_pth'] == contaminant]
file.df = df.copy()
file.overwrite_file()

In [7]:
params  = [
    {'pollutants.def': ('name', [('Diuron', 'solub', 450)])},
    {'pollutants.def': ('name', [('Diuron', 'solub', 600)])},
    {'pollutants.def': ('name', [('Diuron', 'solub', 750)])},
    {'pollutants.def': ('name', [('Diuron', 'solub', 1000)])},
    {'pollutants.def': ('name', [('Diuron', 'solub', 1200)])},
    ]
    #

In [8]:
#check if slurm is running
try:
    cluster = SLURMCluster(
        cores=7,
        memory='24GB'
    )
    cluster.scale(1)
    client = Client(cluster)

except FileNotFoundError as e:    
    print('aaaaaa')
    client = Client()
    

Task exception was never retrieved
future: <Task finished name='Task-26' coro=<_wrap_awaitable() done, defined at /home/zephol/anaconda3/envs/pyswat/lib/python3.10/site-packages/distributed/deploy/spec.py:124> exception=FileNotFoundError(2, 'No such file or directory')>
Traceback (most recent call last):
  File "/home/zephol/anaconda3/envs/pyswat/lib/python3.10/site-packages/distributed/deploy/spec.py", line 125, in _wrap_awaitable
    return await aw
  File "/home/zephol/anaconda3/envs/pyswat/lib/python3.10/site-packages/distributed/deploy/spec.py", line 74, in _
    await self.start()
  File "/home/zephol/anaconda3/envs/pyswat/lib/python3.10/site-packages/dask_jobqueue/core.py", line 411, in start
    out = await self._submit_job(fn)
  File "/home/zephol/anaconda3/envs/pyswat/lib/python3.10/site-packages/dask_jobqueue/core.py", line 394, in _submit_job
    return self._call(shlex.split(self.submit_command) + [script_filename])
  File "/home/zephol/anaconda3/envs/pyswat/lib/python3.10

In [9]:
from dask.distributed import Client

In [10]:
dir_parallel = cwd / 'data' / 'txtinouts' / 'tmp_parallel'
results = reader.run_parallel_swat(params, [], n_workers=5, dir = dir_parallel)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
results

[PosixPath('/mnt/c/Users/jsalo/Desktop/ICRA/pyswat/data/txtinouts/tmp_parallel/tmp040t92ux'),
 PosixPath('/mnt/c/Users/jsalo/Desktop/ICRA/pyswat/data/txtinouts/tmp_parallel/tmp7yf4iyzg'),
 PosixPath('/mnt/c/Users/jsalo/Desktop/ICRA/pyswat/data/txtinouts/tmp_parallel/tmp7u0z7mip')]